The current problem we're facing is

> The simulations run too slow. An average iteration in `Net` below for a classical layer takes ~10ns whereas for a quantum layer it takes ~0.5s, that too parallelized to (8/16 threads)

This is most likely since most of Qiskit is written in Python and not in C-family/Fortran top to bottom. In fact the only C layer in all of Qiskit is the `Aer` package which is why it takes 0.5s and not something like 15s.

There is no way I know of so far of circumventing this problem. the following are a few considerations:
- Using **TorchQuantum**: TorchQuantum does not support Apple Silicon installations (i am using an M2 laptop). The issue is tracked [here](https://github.com/mit-han-lab/torchquantum/issues/98)
- Use the **GPU**: The current Benchmark for M2 CPU is faster than the Colab GPU
- Using **Apple Silicon** to its full extent: While pytorch is already ready for it, Qiskit is not. The issue is tracked [here](https://github.com/Qiskit/qiskit-aer/issues/1762). With full Apple Silicon support, we can use the M2 to basically as much power as the same order of magnitude as Titan
- Using a **Different Algorithm**: See [QCNN.ipynb](./QCNN.ipynb)
- Using **Runtime Primitives**: The `EstimatorQNN` class actually returns the energy levels measured in various ways. The circuit is not learning when using those/I don't know how to use it (since I can't find a lot of examples online)
- Using `TorchConnector`: There is no speed/learning benefit. Under the hood it uses the same `EstimatorQNN` class

### The situation
We know for a fact this model works because under various configurations we are seeing learning happening. For $(0,1)$ case it is almost a perfect classifier. For $(0,1,3,6)$ case for small training size it is random but as the trainset becomes larger it starts becoming more and more better than random (I was able to reach ~40%, perfectly random is ~25%)

It stands to reason if Qiskit were faster we would see the same thing happening for the full MNIST dataset.

In [39]:
from torch.autograd import Function
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch

from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from time import time

from utils import gtt

n_train = 100
n_test = 10
qubits = 13
shots = 1024

# train_loader, test_loader = gtt(n_train, [i for i in range(10)])
train_loader, test_loader = gtt(n_train, [0, 1, 2])

In [70]:
# NC4 from I, X, Y, Z
paulis = ["I", "X", "Y", "Z"]
def select(n):
    if n == 0:
        return [""]
    if n == 1:
        return ["I", "X", "Y", "Z"]
    return [a+b for a in select(n-1) for b in paulis]


class Generator:
    def __init__(self, qubits):
        self.backend = Estimator()
        possible = select(5 if qubits > 5 else qubits)
        # TODO: select properly
        possible = np.random.choice(possible, qubits)
        if qubits > 5:
            possible = ["I"*(qubits-5)+p for p in possible]
        self.obs = [self.op(p) for p in possible]

        circ = QuantumCircuit(qubits)
        for i in range(qubits):
            circ.h(i)
            t = Parameter('t'+str(i))
            circ.cx(i, (i+1)%qubits)
            circ.rx(t, i)
        circ.measure_all()
        self.circuit = circ

    def op(self, p):
        return SparsePauliOp.from_list([(p, 1)])

    def run(self, inputs): # Runs a circuit
        qc = self.circuit.assign_parameters(inputs)
        result = self.backend.run(
            [qc]*len(self.obs),
            self.obs,
            shots=shots,
        ).result()
        return torch.tensor(result.values)

In [71]:
class HybridFunction(Function):
    def forward(ctx, input, quantum_circuit):
        ctx.save_for_backward(input)
        ctx.quantum_circuit = quantum_circuit

        results = []
        for i in range(len(input)):
            expz = ctx.quantum_circuit.run(input[i].tolist())
            results.append(torch.tensor(np.array([expz])))

        return torch.stack(results).squeeze(1)

    def backward(ctx, grad_output):
        inputs = np.array(ctx.saved_tensors[0])
        shift = np.ones(inputs.shape) * np.pi/2

        shiftr = inputs + shift
        shiftl = inputs - shift

        gradients = []
        for i in range(len(inputs)):
            expr = ctx.quantum_circuit.run(shiftr[i])
            expl = ctx.quantum_circuit.run(shiftl[i])

            gradient = torch.tensor(expr - expl)
            gradients.append(gradient)
        # endfor

        gradients = torch.stack(gradients).squeeze(1)
        return gradients * grad_output.float(), None, None

class Hybrid(nn.Module):
    def __init__(self):
        super(Hybrid, self).__init__()
        self.quantum_circuit = Generator(13)
    # end

    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit)
    # end

In [72]:
from time import perf_counter
perfs = []
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=4)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=4)
        out_conv1 = F.max_pool2d(self.conv1(torch.rand(1,1,28,28)), 2);
        out_conv2 = F.max_pool2d(self.conv2(out_conv1), 2)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(out_conv2.view(1,-1).shape[1], qubits)
        self.hybrid = Hybrid()
        out_hybrid = self.hybrid(self.fc1(out_conv2.view(1,-1)))
        self.fc2 = nn.Linear(out_hybrid.shape[1], 10)

    def forward(self, x):
        layer = []
        s = perf_counter()
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x) # NO RELU, pass as-is
        x = self.hybrid(x).type(torch.FloatTensor)
        x = self.fc2(x)
        perfs.append(layer)
        return x

model = Net()

/var/folders/mt/y75db3512zv_8r60rn4pb82h0000gn/T/ipykernel_75047/2635976318.py:13: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstmator from qiskit.primitives.
  self.backend = Estimator()


In [73]:
def normalise(np_mat):
  sums = np_mat.sum(axis=1)
  return np_mat / sums[:, np.newaxis]

In [74]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

epochs = 20 # for now 20
loss_list = [3]

model.train()

targets = []
for epoch in range(epochs):
    total_loss = []
    times = []
    for batch_idx, (data, target) in enumerate(train_loader):
        now = time()
        # optimizer.zero_grad()
        output = model(data)
        targets.append(target)

        loss = loss_func(output, target) # Loss

        loss.backward()
        optimizer.step()

        total_loss.append(loss.item())
        times.append(time() - now)

    print(f"Avg Itr Time: {np.round(np.average(times),1)}s x {len(times)} itrs = {np.round(np.sum(times)/60,1)}min")
    loss_list.append(sum(total_loss)/len(total_loss))

    # process perfs
    perfs2 = normalise(np.array(perfs).astype(np.float16))
    colsums = perfs2.sum(axis=0)
    print(f"Perfs: {colsums}")
    print(f'Training [{100. * (epoch + 1) / epochs:.0f}%]\tLoss: {loss_list[-1]:.4f}')

    diff = np.abs(loss_list[-1] - loss_list[-2]) /loss_list[-1]

    if diff <= 0.001:
        break;
    # endfor
# endfor

/var/folders/mt/y75db3512zv_8r60rn4pb82h0000gn/T/ipykernel_75047/4015201048.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gradient = torch.tensor(expr - expl)


Avg Itr Time: 13.3s x 5 itrs = 1.1min
Perfs: [2.232e-03 2.498e+00 2.500e+00]
Training [5%]	Loss: 2.2713
Avg Itr Time: 12.8s x 5 itrs = 1.1min
Perfs: [4.036e-03 5.000e+00 5.000e+00]
Training [10%]	Loss: 2.2644
Avg Itr Time: 12.0s x 5 itrs = 1.0min
Perfs: [0.007904 7.5      7.5     ]
Training [15%]	Loss: 2.2568
Avg Itr Time: 12.5s x 5 itrs = 1.0min
Perfs: [ 0.010414 10.       10.      ]
Training [20%]	Loss: 2.2522
Avg Itr Time: 12.3s x 5 itrs = 1.0min
Perfs: [ 0.0125 12.5    12.5   ]
Training [25%]	Loss: 2.2394
Avg Itr Time: 13.9s x 5 itrs = 1.2min
Perfs: [1.467e-02 1.500e+01 1.500e+01]
Training [30%]	Loss: 2.2326
Avg Itr Time: 12.5s x 5 itrs = 1.0min
Perfs: [ 0.02032 17.5     17.5    ]
Training [35%]	Loss: 2.2256
Avg Itr Time: 13.9s x 5 itrs = 1.2min
Perfs: [ 0.02716 20.      20.     ]
Training [40%]	Loss: 2.2163
Avg Itr Time: 10.7s x 5 itrs = 0.9min
Perfs: [ 0.0293 22.5    22.5   ]
Training [45%]	Loss: 2.2059
Avg Itr Time: 197.8s x 5 itrs = 16.5min
Perfs: [ 0.0317 25.     25.    ]
Trai

In [ ]:
model.eval() # Set the model to evaluation mode
with torch.no_grad(): # Don't compute gradients
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader): # Loop over the test set
        output = model(data)

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = loss_func(output, target)
        total_loss.append(loss.item())

    print('Performance on test data:\n\tLoss: {:.4f}\n\tAccuracy: {:.1f}%'.format(
        sum(total_loss) / len(total_loss),
        correct / len(test_loader) * 100)
        )

Performance on test data:
	Loss: 2.4200
	Accuracy: 0.0%


In [ ]:
from matplotlib import pyplot as plt
n_samples_show = 6
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

model.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        if count == n_samples_show:
            break
        output = model(data)
        print(output)

        pred = output.argmax(dim=1, keepdim=True)

        axes[count].imshow(data[0].numpy().squeeze(), cmap='gray')

        axes[count].set_xticks([])
        axes[count].set_yticks([])
        axes[count].set_title('Predicted {}'.format(pred.item()))

        count += 1